In [1]:
import numpy as np
import datetime
import os

In [ ]:
def save_fig(fig_name,fig):
    fig_name = os.path.splitext(fig_name)[0]
    fig.savefig(f"{fig_name}.png", dpi=300, facecolor='w', bbox_inches = "tight")

In [2]:
date_format = '%Y-%m-%d'  # "%d/%m/%Y"
def str_to_datetime(s):
    element = datetime.datetime.strptime(s, date_format)
    timestamp = datetime.datetime.timestamp(element)
    return(timestamp)

In [3]:
start = 500
end = 3000
start_gimp_px = 100
end_gimp_px = 835

lbls = ["DVC","DVC_Pro","DCVC","TCM","CANF-VC Lite","CANF-VC","Li et al."]
markers = ["p","s","o","^","P","v","<"]
colors = ["g","c","b","m","tab:cyan","tab:purple","y"]
pts_gimp_px = np.array([200,240,295,365,525,675,425])
years = ['2019-01-01','2020-01-01','2021-01-01','2021-01-01','2022-01-01','2022-01-01','2022-01-01']

In [4]:
kmacperpx = ((pts_gimp_px - start_gimp_px)*(end-start)/(end_gimp_px-start_gimp_px)) + start

In [5]:
kflopsperpx = 2*kmacperpx

In [6]:
kflopsperpx

array([1680.27210884, 1952.38095238, 2326.53061224, 2802.72108844,
       3891.15646259, 4911.56462585, 3210.88435374])

In [7]:
all_systems_compute_flops = kflopsperpx * (1000 * 91701 * 7 * 256 * 256 * 2 * 50)

In [8]:
all_systems_compute_flops

array([7.06857159e+18, 8.21327954e+18, 9.78725297e+18, 1.17904919e+19,
       1.63693237e+19, 2.06619785e+19, 1.35075538e+19])

In [9]:
DAYS_PER_YEAR = 365.2425
HOURS_PER_DAY = 24
MINUTES_PER_HOUR = 60
SECONDS_PER_MINUTE = 60
SECONDS_PER_YEAR = SECONDS_PER_MINUTE * MINUTES_PER_HOUR * HOURS_PER_DAY * DAYS_PER_YEAR

all_systems_datetimes = np.array([str_to_datetime(d) for d in years])
date_intercept_gpu_price_performance = str_to_datetime('2000-01-01')

relative_years = (all_systems_datetimes - date_intercept_gpu_price_performance) / SECONDS_PER_YEAR

tenx_time_gpu_price_performance = 8.17
price_intercept_gpu_price_performance = 7.82 

continuous_price_performance = 10**((1/tenx_time_gpu_price_performance * relative_years) + \
                                    price_intercept_gpu_price_performance)

hardware_replacement_time = 2 * SECONDS_PER_YEAR
hardware_utilization_rate = 0.35

all_systems_training_cost_usd = all_systems_compute_flops / (continuous_price_performance * hardware_replacement_time * hardware_utilization_rate)

In [10]:
all_systems_training_cost_usd

array([22.87839968, 20.05759547, 18.02063977, 21.70907482, 22.74167104,
       28.70539597, 18.76585443])

In [11]:
import matplotlib.pyplot as plt

In [39]:

fig, ax = plt.subplots(nrows=1, ncols=1,figsize=(6.4,4.8))
x = [yr.split("-")[0] for yr in years]
y = 4*all_systems_training_cost_usd
for (m, c, i, j) in zip(markers,colors, x, y):
    ax.plot([i],[j],linestyle="",marker=m,color=c) # 4 lambdas
ax.set_xlabel("Publication Year")
ax.set_ylabel("Training Cost in 2020 US dollars")
for (l, i, j) in zip(lbls, x, y):
    ax.text(i, j+(0 if l == "CANF-VC" else 1), l, ha=('center' if i == '2019' else 'right'))
ax.grid()

In [13]:
# save_fig("USD2020VsYear",fig)

# Energy consumption per pixel in 2020 US dollars

In [14]:
# e = 0.2 * all_systems_training_cost_usd / (91701 * 7 * 256 * 256 * 2 * 50)

In [15]:
# plt.plot(years, e,linestyle="",marker="s")

# Energy consumption per 7 frame sequence in 2020 US dollars

In [16]:

# e_7frame = 0.2 * all_systems_training_cost_usd / (91701 * 2 * 50)

In [17]:
# plt.plot(years, e_7frame,linestyle="",marker="s")

# Energy consumption per 1 hour of video in 2020 US dollars, assuming 60fps

In [18]:
# e_1min = 60 * 60 * 60 * 0.2 * all_systems_training_cost_usd / (91701 * 7 * 2 * 50)

In [19]:
# plt.plot(years, e_1min,linestyle="",marker="s")

# Energy consumption per 1 hour of video in kWh, assuming 60fps

In [20]:
# e_kWh = 60 * 60 * 60 * 0.2 * all_systems_training_cost_usd / (0.13 * 91701 * 7 * 2 * 50)

In [21]:
# plt.plot(years, e_kWh,linestyle="",marker="s")

# Energy consumption per pixel in J

In [22]:
# e_J = 3.6e+6 * 0.2 * all_systems_training_cost_usd / (0.13 * 91701 * 7 * 256 * 256 * 2 * 50) 

In [23]:
# plt.plot(years, e_J,linestyle="",marker="s")

# Energy consumption per pixel in microJ

In [24]:
import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    "legend.fontsize": 14,
    "xtick.labelsize": 14,
    "ytick.labelsize": 14,
    "pgf.preamble": [r"\usepackage{siunitx}"]})

/usr/lib/python3.9/_collections_abc.py:941: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


In [25]:
e_microJ = 1e+6 * 3.6e+6 * 0.2 * all_systems_training_cost_usd / (0.13 * 91701 * 7 * 256 * 256 * 2 * 50) 

In [40]:

fig, ax = plt.subplots(nrows=1, ncols=1,figsize=(6.4,4.8))
y = e_microJ
for (m, c, i, j) in zip(markers,colors, x, y ):
    ax.plot([i],[j],linestyle="",marker=m,color=c)
ax.set_xlabel("Publication Year", fontsize=15)
ax.set_ylabel("$\SI{}{\mu\joule}$ per pixel", fontsize=15)
for (l, i, j) in zip(lbls, x, y):
    ax.text(i, j+(0 if l == "CANF-VC" else 1), l, ha=('center' if i == '2019' else 'right'))
ax.grid()

In [27]:
# save_fig("MicroJoulesPerPixelVsYear",fig)

# Energy consumption per pixel in Wh

In [28]:
# e_Wh = 1000 * 0.2 * all_systems_training_cost_usd / (0.13 * 91701 * 7 * 256 * 256 * 2 * 50) 

In [29]:
# plt.plot(years, e_Wh,linestyle="",marker="s")

# Energy consumption per 1 min of video in Wh, assuming 60fps

A cellphone would last for 18 Wh / 2Wh per min = 9 min

In [30]:
e_Wh_1min = 60 * 60 * 1000 * 0.2 * all_systems_training_cost_usd / (0.13 * 91701 * 7 * 2 * 50) 

In [41]:
fig, ax = plt.subplots(nrows=1, ncols=1,figsize=(6.4,4.8))
y = e_Wh_1min
for (m, c, i, j) in zip(markers,colors, x, y ):
    ax.plot([i],[j],linestyle="",marker=m,color=c)
ax.set_xlabel("Publication Year", fontsize=15)
ax.set_ylabel("Wh per 1 min of video", fontsize=15)
for (l, i, j) in zip(lbls, x, y):
    ax.text(i, j+(0 if l == "CANF-VC" else 0.05), l, ha=('center' if i == '2019' else 'right'))
ax.grid()

In [32]:
# save_fig("WhPerMinOfVideoVsYear",fig)

In [38]:
e_microJ * 256 * 256 * 60 * 60 * 1000 / (1e+6 * 3.6e+6)

array([1.97397967, 1.73059682, 1.55484549, 1.87308872, 1.96218253,
       2.47674089, 1.61914363])

In [35]:
e_Wh_1min

array([1.97397967, 1.73059682, 1.55484549, 1.87308872, 1.96218253,
       2.47674089, 1.61914363])